In [ ]:
import pandas as pd
import numpy as np

import os
from tqdm import tqdm
import json
import shutil

# image data
from PIL import Image
from glob import glob
import cv2
import pydicom

In [ ]:
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential
# from keras.utils import np_utils
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,Activation, Flatten, Dropout, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import regularizers, optimizers
import tensorflow as tf
from tensorflow.keras.applications import ResNet50,VGG16,ResNet101, VGG19, DenseNet201,   MobileNetV2 # ,EfficientNetB4
from tensorflow.keras.applications import resnet, vgg16 , vgg19, densenet,  mobilenet_v2 # ,efficientnet
from tensorflow.keras import Model

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import scipy

import PIL
import os

In [ ]:
gpus =tf.config.experimental.list_physical_devices('GPU')
if gpus: 
    # tensorflow가 첫번째 gpu만 사용하도록 제한
    try: 
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    except RuntimeError as e: 
        print(e)

In [ ]:
tf.test.is_built_with_gpu_support()

In [ ]:
ART_train_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_total_Shuffled_TrainTest/TRAIN/ART/'
ART_test_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_total_Shuffled_TrainTest/TEST/ART/'
PRE_train_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_total_Shuffled_TrainTest/TRAIN/PRE/'
PRE_test_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_total_Shuffled_TrainTest/TEST/PRE/'
GCCT_train_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_total_Shuffled_TrainTest/TRAIN/GCCT/'
GCCT_test_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_total_Shuffled_TrainTest/TEST/GCCT/'

In [ ]:
lst = glob(ART_train_path)
lst.sort()
lst

In [ ]:
test = Image.open(ART_train_path + lst[1])
test

In [ ]:
# height, width, target height, target width
import matplotlib.pyplot as plt
test = Image.open(ART_train_path + lst[1])
test_cropped = tf.image.crop_to_bounding_box(test, 150,110,256,256)
plt.imshow(test_cropped)

In [ ]:
# 02_01_0001 

for i in range(0, 100, 10): 
    plt.figure(figsize = (20, 20))
    test = Image.open(ART_train_path + lst[i])
    test_cropped = tf.image.crop_to_bounding_box(test, 150,110,256,256)
    for j in range(i + 1, i + 10): 
        tmp = Image.open(ART_train_path + lst[j])
        tmp_cropped = tf.image.crop_to_bounding_box(tmp, 150,110,256,256)
        test_cropped = np.concatenate((test_cropped, tmp_cropped), axis = 1)
    plt.imshow(test_cropped)

In [ ]:
# left kidney ymax, xmin
import matplotlib.pyplot as plt
test_cropped = tf.image.crop_to_bounding_box(test, 355, 298, 32,32)
plt.imshow(test_cropped)

In [ ]:
LABEL_path = '/home/ncp/workspace/202002n035/035.신장암 진단을 위한 의료 영상 데이터/01.데이터/신장암2/1.Training/라벨링데이터/'
people = os.listdir(LABEL_path)
file_path = '/home/ncp/workspace/202002n035/035.신장암 진단을 위한 의료 영상 데이터/01.데이터/신장암2/1.Training/원천데이터/'
header_path = file_path + '02_01_0001/' + '02_01_0001_ART/'

In [ ]:
# 02_01_0001 - 47
xy_leftMass_art = []
ppl = '02_01_0001'
ppl_ARTlabel_path = LABEL_path + ppl + '/' + '02_01_0001_ART.json'.format(ppl)
header = pydicom.dcmread(header_path + '0047.dcm', stop_before_pixels = True)
h_num = header.InstanceNumber
total_len = len(os.listdir(header_path))

with open(ppl_ARTlabel_path) as f: 
    ART_json = json.load(f)
    length = len(ART_json['roi_list']['left mass'])
    z_list = []
    for i in range(length): 
        z_list.append(ART_json['roi_list']['left mass'][i]['z'])
    z_list = list(set(z_list))
    
    for j in z_list:
        if h_num == total_len - j: 
            for i in range(len(ART_json['roi_list']['left mass'])):
                xy_leftMass_art.append((ART_json['roi_list']['left mass'][i]['x'], ART_json['roi_list']['left mass'][i]['y']))

In [ ]:
z_list

In [ ]:
file_path = '/home/ncp/workspace/202002n035/035.신장암 진단을 위한 의료 영상 데이터/01.데이터/신장암2/1.Training/원천데이터/'
header_path = file_path + '02_01_0001/' + '02_01_0001_ART/'
print(len(os.listdir(pathh)))
header = pydicom.dcmread(header_path + '0047.dcm', stop_before_pixels = True)
h_num = header.InstanceNumber
print(h_num)
print(len(os.listdir(pathh)) - 59)

In [ ]:
xy_leftMass_art

In [ ]:
len(xy_leftMass_art)

In [ ]:
for i in range(len(ART_json['roi_list']['left mass'])):
    xy_leftMass_art.append((ART_json['roi_list']['left mass'][i]['x'], ART_json['roi_list']['left mass'][i]['y']))

In [ ]:
ART_json['roi_list']['left mass'][0]['x']

In [ ]:
max(xy_leftMass_art[0])

In [ ]:
min(xy_leftMass_art[1])

### mass 부분만 남기고 나머지는 masking

In [ ]:
path = '/home/ncp/workspace/blocks2/kidneyData_windowing_mass_leftright/'
lst = os.listdir(path + 'TRAIN/ART/LEFT/')
lst.sort()
test = cv2.imread(path + 'TRAIN/ART/LEFT/' + lst[0]) # array
image = Image.open(path + 'TRAIN/ART/LEFT/' + lst[0]) # image

In [ ]:
lst[0]

In [ ]:
clist = []
for i in range(len(xy_leftMass_art)): 
    clist.append(xy_leftMass_art[i])

In [ ]:
clist

In [ ]:
mask = np.zeros((test.shape[0], test.shape[1]))
pts = np.array(clist)

In [ ]:
pts

In [ ]:
import cv2

In [ ]:
cv2.fillConvexPoly(mask, pts, 1)

In [ ]:
mask = mask.astype(np.bool)

In [ ]:
out=np.zeros_like(test)

In [ ]:
out[mask] = test[mask]

In [ ]:
plt.figure(figsize = (20,20))
plt.imshow(out)
plt.imsave('test.png', out, cmap = 'gray')

### 전체에 적용

In [ ]:
# create ART_train_label dataframe

LABEL_path = '/home/ncp/workspace/202002n035/035.신장암 진단을 위한 의료 영상 데이터/01.데이터/신장암2/1.Training/라벨링데이터/'
people = os.listdir(LABEL_path) 
ART_train_label = pd.DataFrame()

for ppl in tqdm(people): 
    ppl_ARTlabel_path = LABEL_path + ppl + '/' + '{}_ART.json'.format(ppl)
    with open(ppl_ARTlabel_path) as f: 
        ART_json = json.load(f)
    case_id = ART_json['case_id'][0:10]
    type = ART_json['Clinical Information (Global)']['histologic type']
    
    mass_list = []
    for key in ART_json['roi_list'].keys(): 
        if len(ART_json['roi_list'][key]) > 0 and 'kidney' not in key: # if list is not empty and not kidney - mass exists
            mass_list.append(key)

    data = {'case_id' : [case_id], 'histologic type' : [type], 'where': [mass_list]}
    data = pd.DataFrame(data)
    ART_train_label = pd.concat([ART_train_label, data]).reset_index(drop = True)

In [ ]:
# create ART_val_label dataframe

LABEL_path = '/home/ncp/workspace/202002n035/035.신장암 진단을 위한 의료 영상 데이터/01.데이터/신장암2/2.Validation/라벨링데이터/'
people = os.listdir(LABEL_path) 
ART_val_label = pd.DataFrame()

for ppl in tqdm(people): 
    ppl_ARTlabel_path = LABEL_path + ppl + '/' + '{}_ART.json'.format(ppl)
    with open(ppl_ARTlabel_path) as f: 
        ART_json = json.load(f)
    case_id = ART_json['case_id'][0:10]
    type = ART_json['Clinical Information (Global)']['histologic type']
    
    mass_list = []
    for key in ART_json['roi_list'].keys(): 
        if len(ART_json['roi_list'][key]) > 0 and 'kidney' not in key: # if list is not empty and not kidney - mass exists
            mass_list.append(key)

    data = {'case_id' : [case_id], 'histologic type' : [type], 'where': [mass_list]}
    data = pd.DataFrame(data)
    ART_val_label = pd.concat([ART_val_label, data]).reset_index(drop = True)

In [ ]:
# create PRE_train_label dataframe

LABEL_path = '/home/ncp/workspace/202002n035/035.신장암 진단을 위한 의료 영상 데이터/01.데이터/신장암2/1.Training/라벨링데이터/'
people = os.listdir(LABEL_path) 
PRE_train_label = pd.DataFrame()

for ppl in tqdm(people): 
    ppl_PRElabel_path = LABEL_path + ppl + '/' + '{}_PRE.json'.format(ppl)
    with open(ppl_PRElabel_path) as f: 
        PRE_json = json.load(f)
    case_id = PRE_json['case_id'][0:10]
    type = PRE_json['Clinical Information (Global)']['histologic type']
    
    mass_list = []
    for key in PRE_json['roi_list'].keys(): 
        if len(PRE_json['roi_list'][key]) > 0 and 'kidney' not in key: # if list is not empty and not kidney - mass exists
            mass_list.append(key)

    data = {'case_id' : [case_id], 'histologic type' : [type], 'where': [mass_list]}
    data = pd.DataFrame(data)
    PRE_train_label = pd.concat([PRE_train_label, data]).reset_index(drop = True)

In [ ]:
# create PRE_val_label dataframe

LABEL_path = '/home/ncp/workspace/202002n035/035.신장암 진단을 위한 의료 영상 데이터/01.데이터/신장암2/2.Validation/라벨링데이터/'
people = os.listdir(LABEL_path) 
PRE_val_label = pd.DataFrame()

for ppl in tqdm(people): 
    ppl_PRElabel_path = LABEL_path + ppl + '/' + '{}_PRE.json'.format(ppl)
    with open(ppl_PRElabel_path) as f: 
        PRE_json = json.load(f)
    case_id = PRE_json['case_id'][0:10]
    type = PRE_json['Clinical Information (Global)']['histologic type']
    
    mass_list = []
    for key in PRE_json['roi_list'].keys(): 
        if len(PRE_json['roi_list'][key]) > 0 and 'kidney' not in key: # if list is not empty and not kidney - mass exists
            mass_list.append(key)

    data = {'case_id' : [case_id], 'histologic type' : [type], 'where': [mass_list]}
    data = pd.DataFrame(data)
    PRE_val_label = pd.concat([PRE_val_label, data]).reset_index(drop = True)

In [ ]:
ART_train_label['histologic type'].unique()

In [ ]:
# create new label in label_df
    # cc, chr, pp -> RCC
    # AML
    # ONCO
# (07.28 수정 반영) RCC // AML + onco 로 binary classification

def new_label(df): 
    df['label'] = df['histologic type']
    df.loc[df['label'].isin(['cc', 'chr', 'pp']) == True, 'label'] = 'RCC'
    df.loc[df['label'] == 'AML', 'label'] = 'AML + onco'
    df.loc[df['label'] == 'onco', 'label'] = 'AML + onco'
    return df

ART_train_label = new_label(ART_train_label)
PRE_train_label = new_label(PRE_train_label)
ART_val_label = new_label(ART_val_label)
PRE_val_label = new_label(PRE_val_label)

In [ ]:
# left only, right only 표기하고, left right 두개 다 있으면 both로 표기

def left_right(df): 
    for id in df['case_id']: 
        left = 0
        right = 0
        where = df.loc[df['case_id'] == id, 'where']
        for k in where: 
            for item in k: 
                if 'left' in item: 
                    left = 1
                if 'right' in item: 
                    right = 1
        if left == 1 and right == 1: 
            df.loc[df['case_id'] == id, 'location'] = 'both'
        if left == 1 and right == 0: 
            df.loc[df['case_id'] == id, 'location'] = 'left'
        if left == 0 and right == 1: 
            df.loc[df['case_id'] == id, 'location'] = 'right'
        
    return df

ART_train_label = left_right(ART_train_label)
PRE_train_label = left_right(PRE_train_label)
ART_val_label = left_right(ART_val_label)
PRE_val_label = left_right(PRE_val_label)

In [ ]:
ART_train_label